In [1]:
import os
from vosk import Model

MODEL_NAME = "vosk-model-small-ru-0.22"

if not os.path.exists(MODEL_NAME):
    raise FileNotFoundError("Модель Vosk не найдена")
else:
    model = Model(MODEL_NAME)
    print('Модель Vosk успешно инициализирована!')

Модель Vosk успешно инициализирована!


In [2]:
import os
import wave
import json
from vosk import Model, KaldiRecognizer

def transcribe_audio(model, audio_path):
    #audio_path = 'uploads/record_out.wav'
    #audio_path = 'temp.wav'
    print(audio_path)
    try:
        wf = wave.open(audio_path, "rb")
    except Exception as e:
        print(e)
    rec = KaldiRecognizer(model, wf.getframerate())
    print(wf.getframerate())
    
    result = []
    while True:
        data = wf.readframes(2000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            part_result = json.loads(rec.Result())
            result.append(part_result.get("text", ""))
    
    final_result = json.loads(rec.FinalResult())
    result.append(final_result.get("text", ""))
    print(result)
    return " ".join(result)

In [3]:
text = transcribe_audio(model, r'M:\app_project\uploads\emotions_test.wav')
print(text)

M:\app_project\uploads\emotions_test.wav
48000
['ужасный не красивый плохой злой', 'хороший отличный классный добрый']
ужасный не красивый плохой злой хороший отличный классный добрый


In [22]:
text = 'Angry bad awful good'

In [4]:
from rusentiment import RuSentiment

rs = RuSentiment()
text = "Я в восторге от этого кафе!"
result = rs.predict(text)

print(result)  # Вывод: ['positive']

ModuleNotFoundError: No module named 'rusentiment'

In [6]:
from textblob import TextBlob

a = TextBlob("Привет")
a.translate(to='ru')

AttributeError: 'TextBlob' object has no attribute 'translate'

In [10]:
inputs

NameError: name 'inputs' is not defined

In [14]:
emotions = ['негативный', 'нейтральный', 'позитивный']

In [ ]:
async function analyzeEmotions(text) {
            // Показываем индикатор загрузки
            loadingIndicator.classList.remove('hidden');
            
            try {
                // Отправляем текст на сервер для анализа
                const response = await fetch('http://localhost:5001/analyze_sentiment', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({ text: text })
                });
                
                if (!response.ok) {
                    throw new Error(`Ошибка сервера: ${response.status}`);
                }
                
                const data = await response.json();

                console.log(data);
                
                // Обновляем интерфейс с полученными данными
                updateEmotionBars({
                    happy: Math.round(data.positive * 100),
                    sad: Math.round(data.negative * 100),
                    angry: Math.round((data.neutral * 100) / 2) // Часть нейтрального считаем как "злость"
                });
                
            } catch (error) {
                console.error('Ошибка анализа эмоций:', error);
                showAlert('Не удалось проанализировать эмоции', 'error');
            } finally {
                loadingIndicator.classList.add('hidden');
            }
        }

        // Цветовая маркировка текста по эмоциям
        function colorizeText(text) {
            // Словари эмоциональных слов (можно расширить)
            const happyWords = ['радость', 'счастье', 'ура', 'хорошо', 'прекрасно', 'люблю', 'нравится', 'отлично', 'восторг', 'удовольствие', 'весел', 'рад', 'смех'];
            const sadWords = ['грусть', 'печаль', 'плохо', 'тоска', 'одиночество', 'слезы', 'обида', 'разочарование', 'горе', 'жаль', 'грустн', 'плач', 'несчаст'];
            const angryWords = ['злость', 'ненавижу', 'бесит', 'раздражает', 'отвратительно', 'гнев', 'ужасно', 'кошмар', 'ярость', 'возмущение', 'злюсь', 'сердит', 'бешен'];

            // Разбиваем текст на слова
            const words = text.split(/(\s+)/);

            // Анализируем каждое слово
            const coloredWords = words.map(word => {
                const cleanWord = word.toLowerCase().replace(/[.,!?]/g, '');

                if (happyWords.some(happyWord => cleanWord.includes(happyWord))) {
                    return `<span class="word happy">${word}<span class="emotion-tag tag-happy">радость</span></span>`;
                }
                else if (sadWords.some(sadWord => cleanWord.includes(sadWord))) {
                    return `<span class="word sad">${word}<span class="emotion-tag tag-sad">грусть</span></span>`;
                }
                else if (angryWords.some(angryWord => cleanWord.includes(angryWord))) {
                    return `<span class="word angry">${word}<span class="emotion-tag tag-angry">злость</span></span>`;
                }
                else {
                    return `<span class="word neutral">${word}</span>`;
                }
            });

            return coloredWords.join('');
        }

In [ ]:
@app.route('/analyze_sentiment', methods=['POST'])
def analyze_sentiment():
    text = request.json.get('text', '')
    
    # Анализ всего текста
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    proba = torch.softmax(outputs.logits, dim=1).numpy()[0]
    
    return jsonify({
        'negative': float(proba[0]),
        'neutral': float(proba[1]),
        'positive': float(proba[2])
    })

@app.route('/analyze_words', methods=['POST'])
def analyze_words():
    text = request.json.get('text', '')
    words = list(set(word.strip(".,!?") for word in text.split() if word.strip(".,!?")))
    
    results = []
    for word in words:
        inputs = tokenizer(word, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        proba = torch.softmax(outputs.logits, dim=1).numpy()[0]
        sentiment = ['negative', 'neutral', 'positive'][np.argmax(proba)]
        results.append({
            'word': word,
            'sentiment': sentiment,
            'confidence': float(np.max(proba))
        })
    
    return jsonify(results)

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

model_name = "cointegrated/rubert-tiny-sentiment-balanced"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def get_word_sentiment(word):
    inputs = tokenizer(word, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    proba = torch.softmax(outputs.logits, dim=1).numpy()[0]
    return emotions[np.argmax(proba)]


text = "Плохо нормально!"
for word in text.split():
    print(get_word_sentiment(word))

негативный
позитивный


In [25]:
from textblob import TextBlob

emotion_colors = {
    "радость": "#FFD700",
    "грусть": "#4682B4",
    "гнев": "#FF4500",
    "страх": "#800080",
    "нейтрально": "#C0C0C0"
}

def analyze_emotion(word):
    blob = TextBlob(word)
    sentiment = blob.translate(from_lang='ru', to='en').sentiment.polarity
    print(word, sentiment)
    
    if sentiment > 0.3:
        return "радость"
    elif sentiment < -0.3:
        return "гнев"
    else:
        return "нейтрально"

def colorize_text(text):
    words = text.split()
    colored_text = []
    for word in words:
        emotion = analyze_emotion(word)
        color = emotion_colors.get(emotion, "#C0C0C0")
        colored_text.append(f'<span style="color:{color}">{word}</span>')
    return " ".join(colored_text)

colored_text = colorize_text(text)
print(colored_text)

AttributeError: 'TextBlob' object has no attribute 'translate'

In [95]:
from pydub import AudioSegment
audio = AudioSegment.from_file(r'uploads/record_out.wav')
audio = audio.set_frame_rate(48000).set_channels(1).set_sample_width(2)

raw_data = audio.raw_data
    
rec = KaldiRecognizer(model, 48000)
rec.SetWords(True)  # Если нужно получать слова с таймингами

result = []

# Обрабатываем данные блоками по 4000 байт (как в вашем оригинальном коде)
for i in range(0, len(raw_data), 4000):
    data = raw_data[i:i+4000]
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        part_result = json.loads(rec.Result())
        result.append(part_result.get("text", ""))

# Получаем финальный результат
final_result = json.loads(rec.FinalResult())
result.append(final_result.get("text", ""))
print(result)

['юля', 'юля']


In [92]:
import wave

with wave.open("uploads/record_out.wav", "rb") as wav_file:
    # Основные параметры
    params = wav_file.getparams()
    print("Параметры WAV-файла:")
    print(f"Количество каналов: {params.nchannels}")
    print(f"Ширина сэмпла (байт): {params.sampwidth}")
    print(f"Частота дискретизации (Гц): {params.framerate}")
    print(f"Количество кадров: {params.nframes}")
    print(f"Тип сжатия: {params.comptype}")
    print(f"Имя компрессии: {params.compname}")
    
    # Дополнительно можно получить длительность в секундах
    duration = params.nframes / params.framerate
    print(f"Длительность (сек): {duration:.2f}")

Параметры WAV-файла:
Количество каналов: 1
Ширина сэмпла (байт): 4
Частота дискретизации (Гц): 16000
Количество кадров: 48000
Тип сжатия: NONE
Имя компрессии: not compressed
Длительность (сек): 3.00


In [93]:
import wave

with wave.open("record_out.wav", "rb") as wav_file:
    # Основные параметры
    params = wav_file.getparams()
    print("Параметры WAV-файла:")
    print(f"Количество каналов: {params.nchannels}")
    print(f"Ширина сэмпла (байт): {params.sampwidth}")
    print(f"Частота дискретизации (Гц): {params.framerate}")
    print(f"Количество кадров: {params.nframes}")
    print(f"Тип сжатия: {params.comptype}")
    print(f"Имя компрессии: {params.compname}")
    
    # Дополнительно можно получить длительность в секундах
    duration = params.nframes / params.framerate
    print(f"Длительность (сек): {duration:.2f}")

Параметры WAV-файла:
Количество каналов: 1
Ширина сэмпла (байт): 2
Частота дискретизации (Гц): 48000
Количество кадров: 273600
Тип сжатия: NONE
Имя компрессии: not compressed
Длительность (сек): 5.70


In [88]:
import librosa
wf, sr = librosa.load(r'record_out.wav')
print(wf, sr)

[-5.8686012e-10 -6.5592720e-10  9.3923325e-10 ...  3.3134042e-04
 -1.6097634e-04 -1.9135085e-04] 22050
